In [1]:
!pip install transformers datasets huggingface_hub fsspec==2024.10.0 -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is incompatible.
s3fs 2024.9.0 requires fsspec==2024.9.0.*, but you have fsspec 2024.10.0 which is incompatible.


## Hugging face 사용 이유
- 여러 회사들이 개발한 모델이나 제공하는 데이터셋을 편리하게 사용하는 인터페이스 제공

In [2]:
from transformers import AutoModel, AutoTokenizer

### Bert 모델 활용 준비
- https://huggingface.co/google-bert/bert-base-uncased

In [3]:
bert_model = AutoModel.from_pretrained('google-bert/bert-base-uncased')
bert_tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
type(bert_model), type(bert_tokenizer)

(transformers.models.bert.modeling_bert.BertModel,
 transformers.models.bert.tokenization_bert_fast.BertTokenizerFast)

In [5]:
text = 'What is Huggingface Transformers ?'
encoded_input = bert_tokenizer(text, return_tensors='pt') # 학습 완료된 형태에 벡터로 결과 리턴
bert_out = bert_model(**encoded_input)

In [6]:
# type(encoded_input), encoded_input
type(bert_out), bert_out

(transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions,
 BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3009,  0.0158,  0.0698,  ..., -0.3406,  0.5976,  0.5820],
          [-0.1109,  0.0754, -0.1906,  ...,  0.2970,  0.4278, -0.0391],
          [-0.5813, -0.0042,  0.4034,  ..., -0.2549,  0.2216,  0.8121],
          ...,
          [ 0.9971,  0.3301, -0.0688,  ..., -0.4873,  0.0168, -0.0345],
          [-0.2394, -0.0573, -0.5885,  ..., -0.0415,  0.3123, -0.0288],
          [ 0.7884,  0.4039,  0.0217,  ...,  0.3869, -0.4785, -0.4116]]],
        grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.7247e-01, -3.1464e-01, -5.2733e-01,  7.3685e-01,  1.8460e-01,
          -1.8295e-01,  8.9985e-01,  2.9762e-01, -3.9314e-01, -9.9994e-01,
           7.1273e-02,  7.2618e-01,  9.7232e-01,  3.2968e-01,  9.1369e-01,
          -7.5043e-01, -3.5754e-01, -5.6530e-01,  2.8968e-01, -6.4040e-01,
           6.0199e-01,  9.9856e-01,  3.8675e-01,  2.9

#### decoding

In [7]:
import torch

# 출력값 접근하기
token_embeddings = bert_out.last_hidden_state  # (1, seq_len, hidden_dim)
sentence_embedding = bert_out.pooler_output  # (1, hidden_dim)

# 첫 번째 토큰 '[CLS]'의 임베딩 (문장 표현으로 사용 가능)
cls_embedding = token_embeddings[0][0]

# 각 토큰 확인하기
tokens = bert_tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
print("Tokens:", tokens)

# 각 토큰과 그 임베딩 값 출력 (첫 3개만 예시로 확인)
for i, token in enumerate(tokens[:3]):
    print(f"Token: {token}, Embedding: {token_embeddings[0][i][:5].tolist()}")  # 5개 값만 출력


Tokens: ['[CLS]', 'what', 'is', 'hugging', '##face', 'transformers', '?', '[SEP]']
Token: [CLS], Embedding: [-0.3008577823638916, 0.0157686285674572, 0.06975799053907394, -0.25098690390586853, -0.4629148244857788]
Token: what, Embedding: [-0.11086355149745941, 0.07536714524030685, -0.19062983989715576, -0.3838910758495331, -0.19290482997894287]
Token: is, Embedding: [-0.581269383430481, -0.0041732145473361015, 0.40341517329216003, -0.36582842469215393, 0.26863595843315125]


## 모델 불러오기
- 모델 : 바디와 헤드로 구분

In [8]:
# 바디만 있는 모델
# https://huggingface.co/klue/roberta-base
roberta_model = AutoModel.from_pretrained('klue/roberta-base')

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
roberta_model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [10]:
# 바디와 헤드가 포함된 모델
# https://huggingface.co/SamLowe/roberta-base-go_emotions
from transformers import AutoModelForSequenceClassification
roberta_go_emotions_model = AutoModelForSequenceClassification.from_pretrained('SamLowe/roberta-base-go_emotions')

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [11]:
roberta_go_emotions_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [12]:
# 헤드에 없는 모델을 목적에 의해 불러오기
roberta_classification_model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
roberta_classification_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         